<a href="https://colab.research.google.com/github/megmenegazzi/AMD-project/blob/main/AMD_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Install stuff**

In [ ]:
!pip install pyspark

In [ ]:
!pip install -q findspark

## **Import stuff**

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
import pyspark
type(spark)

In [ ]:
sc = spark.sparkContext

In [ ]:
import os
import findspark
import pandas as pd

## **Load Data**

In [ ]:
os.environ["KAGGLE_USERNAME"] = "margheritamenegazzi"
os.environ["KAGGLE_KEY"] = "75953089094982034e32cf970ea2d0e2"

In [ ]:
!kaggle datasets download ashirwadsangwan/imdb-dataset --unzip

## **Clean Data**

In [ ]:
data = pd.read_csv("title.principals.tsv/title.principals.tsv", sep ="\t")

In [ ]:
data.head()

In [ ]:
data.drop(['ordering', 'category', 'job', 'characters'], axis=1, inplace=True)

In [ ]:
data.rename(columns={"tconst": "title", "nconst": "actor"}, inplace=True)

In [ ]:
data.title = data.title.apply(lambda title : int(title[2:]))
data.actor = data.actor.apply(lambda actor : int(actor[2:]))

In [ ]:
data.head()

## **Load Dataset on Spark**

In [ ]:
sc = spark.sparkContext

In [ ]:
data_rec = data.to_records(index = False, )

In [ ]:
# for i in range(len(data_rec)):
#   for j in range(len(data_rec[i])):
#     data_rec[i][j] = int(data_rec[i][j])

In [ ]:
simple_rdd = sc.parallelize(data_rec[:50000])

In [ ]:
_ = simple_rdd.collect()

In [ ]:
inverted = simple_rdd.map(lambda t : (t[1], t[0]))

In [ ]:
_ = inverted.collect()

In [ ]:
joined = inverted.join(inverted)

In [ ]:
_ = joined.collect()

In [ ]:
filtered = joined.filter(lambda x : x[1][0]!= x[1][1])

In [ ]:
_ = filtered.collect()

In [ ]:
links = filtered.map(lambda x : x[1])

In [ ]:
links.take(2)

In [ ]:
def adj(x,y):
  
#   if type(x) is not list:
#     mylist = []
#     mylist.append(x)
#     x = mylist

# # x = [x] if type(x) is str else x

#   if type(y) is not list:
#     mylist1 = []
#     mylist1.append(y) 
#     y = mylist1 

  for elem in y:
    x.append(elem)
  return x   

In [ ]:
adjacency1 = links.mapValues(lambda v: [v])

In [ ]:
adjacency = adjacency1.reduceByKey(adj)

In [ ]:
adjacency.take(2)

In [110]:
def conn(x):
  k, v = x
  mario = []
  for vi in v:
    luigi = (k, vi, 1/len(v))
    mario.append(luigi)
  return mario 


In [111]:
connection = adjacency.flatMap(conn)

In [112]:
connection.take(2)

[(12, 16, 0.034482758620689655), (12, 29, 0.034482758620689655)]

In [113]:
KL = sorted(connection.map(lambda x : x[0]).distinct().collect()) # keys list

In [114]:
dizionario = dict(zip(KL, range(len(pluto))))

In [119]:
def palle(x):
  k,v,f=x

  k = dizionario[k]
  v = dizionario[v]

  nuovatupla = ((k),(v,f))

  return nuovatupla


fabrizio = connection.map(palle)

In [145]:
import numpy as np
n = len(KL)
page_rank = np.ones(n)/n
old_page_rank = np.ones(n)

In [149]:
fabrizio.mapValues(lambda x : x[1]).collect()

[(11, 0.034482758620689655),
 (11, 0.034482758620689655),
 (11, 0.034482758620689655),
 (11, 0.034482758620689655),
 (11, 0.034482758620689655),
 (11, 0.034482758620689655),
 (11, 0.034482758620689655),
 (11, 0.034482758620689655),
 (11, 0.034482758620689655),
 (11, 0.034482758620689655),
 (11, 0.034482758620689655),
 (11, 0.034482758620689655),
 (11, 0.034482758620689655),
 (11, 0.034482758620689655),
 (11, 0.034482758620689655),
 (11, 0.034482758620689655),
 (11, 0.034482758620689655),
 (11, 0.034482758620689655),
 (11, 0.034482758620689655),
 (11, 0.034482758620689655),
 (11, 0.034482758620689655),
 (11, 0.034482758620689655),
 (11, 0.034482758620689655),
 (11, 0.034482758620689655),
 (11, 0.034482758620689655),
 (11, 0.034482758620689655),
 (11, 0.034482758620689655),
 (11, 0.034482758620689655),
 (11, 0.034482758620689655),
 (15, 0.045454545454545456),
 (15, 0.045454545454545456),
 (15, 0.045454545454545456),
 (15, 0.045454545454545456),
 (15, 0.045454545454545456),
 (15, 0.045454

In [147]:
def l2distance(v, q):
    
    if len(v) != len(q):
        raise ValueError(f'Cannot compute the distance of two vectors of size {len(v)} and {len(q)}')
    
    return sum([(q_el - v_el)**2 for v_el, q_el in zip(v, q)])

In [148]:
tolerance = 0
max_iterations = 5
 
iteration = 0
while(l2distance(old_page_rank, page_rank) >= tolerance and iteration < max_iterations):
    page_rank_values = (fabrizio
                        .mapValues(lambda v: v[1]*page_rank[v[0]])
                        .reduceByKey(lambda a, b: a+b)
                        .sortByKey()
                        .collect()
                       )
    old_page_rank = page_rank
    print(page_rank_values)
    page_rank = np.array([c for (i, c) in page_rank_values])
    
    # we use the nice_print function to show how the page_rank vector
    # evolves over time
    # nice_print(page_rank)
    print(iteration)
    iteration += 1

[(0, 0.00013073604392731074), (1, 0.00013073604392731077), (2, 0.00013073604392731077), (3, 0.00013073604392731077), (4, 0.00013073604392731074), (5, 0.00013073604392731069), (6, 0.00013073604392731085), (7, 0.00013073604392731074), (8, 0.00013073604392731074), (9, 0.0001307360439273108), (10, 0.00013073604392731077), (11, 0.00013073604392731077), (12, 0.0001307360439273108), (13, 0.0001307360439273108), (14, 0.00013073604392731074), (15, 0.00013073604392731074), (16, 0.00013073604392731077), (17, 0.00013073604392731077), (18, 0.00013073604392731077), (19, 0.00013073604392731077), (20, 0.0001307360439273108), (21, 0.0001307360439273108), (22, 0.00013073604392731077), (23, 0.00013073604392731077), (24, 0.0001307360439273108), (25, 0.0001307360439273108), (26, 0.0001307360439273107), (27, 0.00013073604392731077), (28, 0.00013073604392731077), (29, 0.0001307360439273108), (30, 0.00013073604392731077), (31, 0.0001307360439273108), (32, 0.00013073604392731058), (33, 0.00013073604392731058),